# 🌐 Connecting AI Agents to External APIs: Search & Compute

Welcome to this hands-on guide where we'll build a sophisticated AI agent that can connect to the real world. By integrating external APIs, we can empower our agent to answer questions about current events, perform complex calculations, and access structured data, overcoming the limitations of a standard LLM.

### Lab Objectives:
By the end of this lab, you will have:
1.  **Acquired API keys** for Google Search (via SerpAPI) and Wolfram Alpha.
2.  **Built custom tools** for each API, making them available to a LangChain agent.
3.  **Assembled a multi-tool agent** that can intelligently choose between searching the web and performing computations.
4.  **Observed the agent's reasoning** as it decides which tool is best for a given query.
5.  **Added memory** to the agent to enable conversational follow-ups.

### Part 1: Setup and API Key Configuration

#### Step 1.1: Install Required Libraries

We need to install the libraries for LangChain, OpenAI, SerpAPI, and `python-dotenv` for managing our keys. We will **not** use the `wolframalpha` library due to conflicts with notebook environments.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
!pip install -q langchain langchain-openai google-search-results python-dotenv requests

#### Step 1.2: Set Up Your API Keys

This agent requires **three** API keys to function. This code will first attempt to load them from a `.env` file in the current directory. If it can't find the file or the keys within it, it will securely prompt you to enter them.

**To use a `.env` file (recommended for local use):**
1.  Create a file named `.env` in the same directory as this notebook.
2.  Add your keys like this:
    ```
    OPENAI_API_KEY="sk-YourKeyHere"
    SERPAPI_API_KEY="YourSerpApiKeyHere"
    WOLFRAM_ALPHA_APPID="YourWolframAppIdHere"
    ```

In [2]:
import os
import getpass
from dotenv import load_dotenv

# Load environment variables from .env file, if it exists
dotenv_path = os.path.expanduser("~/dev.env")
load_dotenv(dotenv_path)

def get_api_key(key_name: str) -> str:
    """Gets an API key from environment variables or prompts the user for it."""
    key = os.getenv(key_name)
    if not key:
        print(f"Could not find {key_name} in .env file.")
        key = getpass.getpass(f"Please enter your {key_name}: ")
    return key

# Get all three API keys
OPENAI_API_KEY = get_api_key("OPENAI_API_KEY")
SERPAPI_API_KEY = get_api_key("SERPAPI_API_KEY")
WOLFRAM_ALPHA_APPID = get_api_key("WOLFRAM_ALPHA_APPID")

# Set them as environment variables for the libraries to use
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['SERPAPI_API_KEY'] = SERPAPI_API_KEY
os.environ['WOLFRAM_ALPHA_APPID'] = WOLFRAM_ALPHA_APPID

print("✅ API Keys are set for this session.")

✅ API Keys are set for this session.


### Part 2: Creating the Google Search Tool

This tool will allow our agent to search the live web to answer questions about recent events or topics not in its training data. We use SerpAPI as a convenient way to access Google Search results.

#### Step 2.1: Define the Search Function

This Python function takes a search query, sends it to the SerpAPI, and processes the results to return a clean, readable summary of the top hits.

In [3]:
from langchain.tools import Tool
from serpapi import GoogleSearch

def google_search_func(query: str) -> str:
    """Performs a Google search using the SerpAPI and returns a summary of the top results."""
    print(f"\n{'='*10} DEBUG: GOOGLE SEARCH TOOL {'='*10}")
    print(f"Performing Google Search for: '{query}'")
    params = {
        "engine": "google",
        "q": query,
        "api_key": os.environ['SERPAPI_API_KEY'],
        "num": "5" # Request 5 results
    }
    
    try:
        search = GoogleSearch(params)
        results = search.get_dict()
        organic_results = results.get("organic_results", [])
        
        if not organic_results:
            return "No good search results found."
        
        summary_parts = []
        for res in organic_results[:3]:
            title = res.get("title", "No Title")
            snippet = res.get("snippet", "No Snippet")
            summary_parts.append(f"Title: {title}\nSnippet: {snippet}")
        
        final_summary = "\n\n---\n\n".join(summary_parts)
        return final_summary
    except Exception as e:
        return f"Error during Google Search: {e}"

#### Step 2.2: Wrap and Test the Tool

Now we wrap our function in a LangChain `Tool` object.

In [4]:
google_tool = Tool(
    name="google_search",
    func=google_search_func,
    description="Use this tool to find information about recent events, current affairs, or topics on the web. It is good for finding facts and summaries."
)

print("✅ Google Search Tool created.")

✅ Google Search Tool created.


### Part 3: Creating the Wolfram Alpha Tool (Robust Method)

#### Step 3.1: Define the Wolfram Alpha Function using `requests`

This function calls the Wolfram Alpha "Full Results API" endpoint. It requests the output in JSON format and then parses the response to find the most direct answer, usually found in the pod titled "Result" or "Value".

In [5]:
import requests

def wolfram_query_func_requests(query: str) -> str:
    """Sends a query to Wolfram Alpha's Full Results API and returns a plaintext answer."""
    print(f"\n{'='*10} DEBUG: WOLFRAM ALPHA REQUESTS TOOL {'='*10}")
    print(f"Querying Wolfram Alpha for: '{query}'")
    
    app_id = os.environ['WOLFRAM_ALPHA_APPID']
    api_url = "http://api.wolframalpha.com/v2/query"
    
    params = {
        "input": query,
        "format": "plaintext",
        "output": "JSON",
        "appid": app_id
    }
    
    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status() # Raise an error for bad status codes
        data = response.json()
        
        # The API response is complex. We need to parse it carefully.
        query_result = data.get("queryresult", {})
        pods = query_result.get("pods", [])
        
        if query_result.get("success") and pods:
            # Find the most relevant pod (usually 'Result' or 'Value')
            for pod in pods:
                if pod.get("primary"): # The primary pod is usually the main answer
                    subpods = pod.get("subpods", [])
                    if subpods:
                        answer = subpods[0].get("plaintext")
                        print(f"DEBUG: Found primary answer: {answer}")
                        return f"Answer from Wolfram Alpha: {answer}"
            # If no primary pod, return the first pod's answer
            if pods[0].get("subpods"): 
                answer = pods[0]["subpods"][0].get("plaintext")
                print(f"DEBUG: No primary pod, returning first result: {answer}")
                return f"Answer from Wolfram Alpha ({pods[0].get('title')}): {answer}"
        
        return "Wolfram Alpha could not find an answer."

    except requests.exceptions.HTTPError as http_err:
        return f"Error: Wolfram API returned status {http_err.response.status_code}. This often means your AppID is invalid."
    except Exception as e:
        return f"An unexpected error occurred: {e}"

#### Step 3.2: Wrap and Test the New Tool

Now, let's wrap our new, robust function as a LangChain tool.

In [6]:
wolfram_tool = Tool(
    name="wolfram_alpha_query",
    func=wolfram_query_func_requests, # Use our new requests-based function
    description="Use this tool for any questions requiring math, calculations, unit conversions, or scientific data. It is a computational and factual engine."
)

# Let's test the new tool directly
print("--- Testing New Wolfram Alpha Tool Directly ---")
test_result = wolfram_tool.run("What is the distance from the Earth to the Moon in kilometers?")
print("\n--- End of Test ---")
print("\nTool Output:")
print(test_result)

--- Testing New Wolfram Alpha Tool Directly ---

========== DEBUG: WOLFRAM ALPHA REQUESTS TOOL ==========
Querying Wolfram Alpha for: 'What is the distance from the Earth to the Moon in kilometers?'
DEBUG: Found primary answer: 403932 km (kilometers)

--- End of Test ---

Tool Output:
Answer from Wolfram Alpha: 403932 km (kilometers)


### Part 4: Building the Multi-Tool Agent

Now we assemble our agent, giving it access to both the Google Search tool and our new, robust Wolfram Alpha tool.

In [ ]:
from langchain_openai import OpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory

# Initialize the LLM
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-instruct")

# Create the list of tools
tools = [google_tool, wolfram_tool]

# Initialize memory to allow for follow-up questions
# Setting return_messages=True makes the memory object store and return the conversation history as a list of message objects instead of a single string.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# CONVERSATIONAL_REACT_DESCRIPTION is an agent type that combines the step-by-step reasoning of the ReAct framework with memory, allowing it to have a conversation and use tools based on their descriptions.
# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

print("✅ Multi-tool conversational agent created successfully!")

✅ Multi-tool conversational agent created successfully!


/tmp/ipykernel_8007/3383895787.py:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/tmp/ipykernel_8007/3383895787.py:15: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


#### Step 4.1: Test the Agent's Tool Selection

Let's test our fully assembled agent.

In [8]:
# Test A: A question requiring computation
agent.run("What is 25% of 5820?")

/tmp/ipykernel_8007/2629581940.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("What is 25% of 5820?")




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: wolfram_alpha_query
Action Input: What is 25% of 5820?
========== DEBUG: WOLFRAM ALPHA REQUESTS TOOL ==========
Querying Wolfram Alpha for: 'What is 25% of 5820?'
DEBUG: Found primary answer: 1455

Observation: Answer from Wolfram Alpha: 1455
Thought: Do I need to use a tool? No
AI: 25% of 5820 is 1455. Is there anything else you would like to know?

> Finished chain.


'25% of 5820 is 1455. Is there anything else you would like to know?'

In [9]:
# Test B: A question requiring a web search
agent.run("Who is the current prime minister of the United Kingdom?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: google_search
Action Input: Who is the current prime minister of the United Kingdom?
========== DEBUG: GOOGLE SEARCH TOOL ==========
Performing Google Search for: 'Who is the current prime minister of the United Kingdom?'

Observation: Title: Prime Minister of the United Kingdom
Snippet: Prime Minister of the United Kingdom ; Incumbent Keir Starmer. since 5 July 2024 ; Government of the United Kingdom · Prime Minister's Office ...

---

Title: Prime Minister
Snippet: Sir Keir Starmer became Prime Minister on 5 July 2024. Education. Keir attended Reigate Grammar School, before studying Law at the University of Leeds. He went ...

---

Title: Keir Starmer
Snippet: Rishi Sunak · Prime Minister of the United Kingdom 2024–present, Incumbent. Minister for the Civil Service 2024–present. First Lord of the Treasury 2024–present.
Thought: Do I need to use a tool? No
AI: The current prime minister of the Unite

'The current prime minister of the United Kingdom is Keir Starmer. Is there anything else you would like to know?'

In [ ]:
# CONVERSATIONAL_REACT_DESCRIPTION: Remember the previous interactions.
agent.run("When hea was born?")

### Conclusion

Congratulations! You've built a powerful, conversational AI agent that can leverage external APIs to answer a wide range of questions. By diagnosing the library issue and building a more robust tool with `requests`, you've also learned a valuable lesson in debugging and working with external services.

#### Key Takeaways:
1.  **Direct API Calls are a Powerful Debugging Tool**: When a library fails, testing the API directly with `requests` can quickly identify the root cause.
2.  **Robustness over Convenience**: Sometimes, bypassing a convenient but problematic library in favor of a direct implementation is the best solution.
3.  **Agents are Force Multipliers**: By connecting to APIs, agents can provide real-time, factual, and computationally accurate information.